## Tutorial: Using pd_exp to create experiments with Prisoner's Dilemma Tournaments
This notebook walks through two different experimental setups to showcase the code functionality of the `pd_exp` module.

In [ ]:
# Local modules to import
from code import pd_exp
from code.helper_funcs import partitions
from code.settings import player_names, stag

# Other Python modules used
import pandas as pd
from pathlib import Path
import time, json

`pd_exp` was designed to collect data from multiple groups of round-robin tournaments. To keep vocabulary consistent, let me introduce some project terminology:
- _Supergame_: a single PD instance between two players that consists of one or more rounds of play.
- _Tournament_: a round-robin tournament between a select group of players. This concept can also be referred to as a team.
- _System_: a collection of one or more teams.

In the code below, I setup data collection for two different experiments. In experiment 1, we consider how measured outcomes differ within one team of deterministic players in two different game variations: classic PD and Stag Hunt. In experiment 2, we expand on experiment 1 to consider multiple different systems, where each system is comprised of two teams.

#### __Team Metrics__
To measure team outcomes, we use the following metrics: 1) minimum of normalized player scores, 2) average of normalized player scores, and 3) average normalized CC distribution.

##### __The Minumum (Min Score) and Average (Avg Score) of Normalized Player Scores__
The normalized score for a player, $j$, is computed as 
$$
S_j = \sum_{i\neq j} \frac{\text{Score-per-turn against opponent } i}{\text{Total opponents}} = \frac{1}{\text{Total opponents}} \sum_{i\neq j} \frac{S_{ji}}{T_{ji}},
$$
where $S_{ji}$ is player $j$'s score against player $i$ and $T_{ji}$ is the total turns played between player $j$ and $i$.

The minimum and average values across all normalized player scores are taken as aggregate group measures to reflect group welfare.

##### __The Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{1}{N} \sum_{j=1}^N \frac{1}{N-1} \sum_{i \neq j} \frac{CC \text{ count against player } i}{\text{total turns against player }i}
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.


#### __System Metrics__
To measure system outcomes, we extend the above metrics and use six different measures:
1. System Minimum Score (SYS MIN)
2. System Average Score (SYS AVG)
3. Minimum of Team Average Scores (Min of Team Avgs)
4. Average of Team Minimum Scores (Avg of Team Mins)
5. System CC Distribution Average (SYS CC Dist AVG)
6. System CC Distribution Average (SYS CC Dist MIN)

### Experiment 1: Comparing team performance across classic PD and Stag Hunt

In [ ]:
### For experiment 1
# Since this simulation gathers data for systems, I coded systems as ordered-lists of 
# teams. To implement ordered-lists of teams I nested player_names within closed-brackets and then a tuple to create an ordered-list.
# The simulation code allows for processing numerous systems, so I enclose the tournament within another pair
# of closed-brackets and then a tuple.

part_list = tuple([tuple([player_names])]) # 'part' is shorthand for partition

In [ ]:
print(part_list)

In [ ]:
print('Total count of systems: ',len(part_list))  #Should be 5775 for n=12 and k=4
start_time = time.time()

print('Instantiate PD Experiments with different game types')
# To instantiate a PD Experiment, I use the PDExp constructor with at partition list and game-type, which is 
# by default the classic PD)
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)


print('Running experiments and computing data')
# To generate the simulation data and compute the group metrics, I use the run_experiments() method.
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()


print('Saving experiment data')
### For experiment 1
path = 'Data/Experiment1/'
classic_pdExp.save_data(path, 'ClassicPD_test_DELETE')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_test_DELETE')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

### Experiment 1: Data Comparison Across Games

In [ ]:
# Retrieve saved data
classic_dataf1 = pd.read_csv('Data/Experiment1/ClassicPD_test_DELETE_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf1 = pd.read_csv('Data/Experiment1/StagHunt_test_DELETE_RPST_5_1_0_3.csv', index_col=0)

#### _The System ID_
Since these experiments are with unique players, each system is denoted with a unique ID that is comprised of numbers, commas, and underscores. The numbers represent players and determined by the alphabetical order of the player's strategy out of the total set of player strategies in system. The commas delimit players of the same team, and underscores delimit different teams of the same system. Note the system IDs and team compositions below and how they differ between experiment 1 and experiment 2. 

In [ ]:
# I use option_context to customize the limits of the dataframe display
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

In [ ]:
# SIDENOTE: Documentation (i.e. docstring) for code can be retrieved with '?'. Here are some examples.

?pd_exp

In [ ]:
?pd_exp.PdExp

### Experiment 2: Comparing systems of 3 teams across classic PD and Stag Hunt

In [ ]:
### For experiment 2
# To create numerous systems with a partitioned list of teams can take time, so I save the tuple of systems 
# to file immediately after computing all k-partition subsets of a set of players  to avoid having to recompute 
# them at each run.

# Checking for txt file with list of systems
p = Path('partition_list_teams_of5_DEMO.txt')
if p.exists():
   with open(p, "r") as f:
       part_list = json.load(f)
else:
   print('creating partitions and saving to file')
   part_list = list(partitions(player_names,5))
   p.touch()
   with open(p, "w") as f:
       json.dump(part_list, f)

In [ ]:
part_list = part_list[:3] # Let's just consider 3 different systems
print('Total count of systems: ',len(part_list))
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_4x3_12uniq_DEMO')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

### Experiment 2: Data Comparison Across Systems, Teams, and Games

In [ ]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf = pd.read_csv('Data/Experiment2/StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)
stagHunt_dataf = stagHunt_dataf.reset_index(drop=True)

In [ ]:
# The computed results create a large spreadsheet or dataframe. Here is a list of just the column names.
classic_dataf.columns

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team2']])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team2']])

#### How do the systems compare within the same game?

In [ ]:

classic_dataf.loc[:,:'SYS CC Dist MIN']

In [ ]:

stagHunt_dataf.loc[:,:'SYS CC Dist MIN']

#### How did team 1 and 2 performance differ across systems?

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team2','Team2 Avg Score',
       'Team2 Min Score', 'Team2 Avg CC Dist']])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
                       'display.max_colwidth', None):  # more options can be 
                                                       # specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team1 Avg Score',
                            'Team1 Min Score', 'Team1 Avg CC Dist', 'Team2',
                            'Team2 Avg Score', 'Team2 Min Score', 
                            'Team2 Avg CC Dist']])